__講義で使用するファイルについて__

本講義では配列ファイルの扱い方を学びます。使用するファイルは出芽酵母Saccharomyces cerevisiae S288Cのもので  
https://www.ncbi.nlm.nih.gov/assembly/GCF_000146045.2  
から取得した、ゲノム塩基配列（FASTA形式）と遺伝子アノテーションを含んだファイル（GenBank形式およびGFF形式）です。

# クラスの利用

## クラスとオブジェクト

Pythonプログラムの中で扱われる文字列や数値、あるいはリストや辞書といったデータを「オブジェクト」と呼びます。また、データに対して何らかの処理を行う機能を持った関数やオブジェクトの設計図である「クラス」自体もオブジェクトの１つとして扱われます。したがって、Pythonプログラムの中で扱われるさまざまな「モノ」に対しての総称が「オブジェクト」といえます。オブジェクトにはデータの種類に応じて「型」があります。例えば、整数データはintオブジェクト、文字列データはstrオブジェクトと呼ばれ、これらはtype関数を使用することで確認ができます。

In [ ]:
type(10)

In [ ]:
my_str = "Hello world"
type(my_str)

また、各オブジェクトがどのような機能（メソッド）を持っているかはdir関数を使用することで確認できます。


In [ ]:
print(dir(my_str))

strオブジェクトには大文字に変換するためのupperや分割を行うためのsplitといった文字列に対する操作を行うためのメソッドが含まれます。これらを使用するには、変数名の後にピリオドに続けてメソッド名を指定します。

In [ ]:
my_str.upper()

In [ ]:
my_str.split(" ")

通常の関数と同じようにメソッドを呼び出すにはメソッド名の後の括弧に引数を指定します。上記の例で、upperは特に引数を必要としないため括弧内は空欄になっているのに対し、splitの場合には分割に用いる文字を引数に指定しています。なお、何も引数を指定せずにsplitを使用することもでき、その場合にはスペースやタブなど空白とみなせる文字が分割の対象となります。

より複雑な機能やデータをもったオブジェクトを扱うために、拡張モジュールをインポートして定義済みの型（たとえばdatetimeオブジェクトなど）を利用したり、あるいは自分でオブジェクトの設計図（＝型）である「クラス」を定義したりすることができます。Biopythonをインポートすれば塩基配列やアミノ酸配列データを扱うためのSeqオブジェクトが利用可能ですが、ここでは自分でクラスを定義する例題としてFASTA形式で記述された配列データを格納するためのクラスを設計してみます。

下記は１件のFASTA形式の配列データの例です。
```
>gene01 nucleotide sequence of tRNA-Ser
TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT
```
">"で始まる行はタイトル行で、一般に最初の空白までが配列IDを示し、それ以降の文字列は遺伝子の機能名や任意の説明書きが含まれます。2行目が配列データを示します。この例では改行が含まれていませんが、60〜100文字単位で改行が含まれることもあります。

これらの情報を格納するためには、  
・配列ID (id)  
・配列に対しての説明 (description)  
・配列自体のデータ (seq）  
といった情報を含んだクラスを設計することになります。また、この配列データに対しての操作としてGC含量（塩基配列中のGおよびCの割合）を得るためのメソッドを例として定義してみます。
以下がその例です。

In [ ]:
class Fasta:
    def __init__(self, id_, description, seq):
        self.id = id_
        self.description = description
        self.seq = seq
            
    def get_gc_content(self):
        g_count = self.seq.count("G")
        c_count = self.seq.count("C")
        gc_content = ( g_count + c_count ) / len(self.seq)
        return gc_content

ここでクラス内に定義した関数は、単独で用いる関数と区別してメソッドと呼ばれます。最初に定義したメソッド`__init__`は、オブジェクトの設計図であるクラスに実際のデータを格納してオブジェクトを生成する際に実行されるコンストラクタと呼ばれる特殊メソッドです。  
メソッドの第一引数は自身のオブジェクトを指すもので、pythonでは慣例的に"self"が使われます。残りの３つはオブジェクト生成時に与える引数で、与えられた情報はオブジェクトの内部的な変数である`self.id`、`self.description`、`self.seq`にそれぞれ格納されます。なお、配列IDを示す引数は、組み込み関数である"id"と区別するために末尾に"_"を付けて"id_"としていますが、"_"を付けなくても動作します。  
それでは、実際にデータを与えて配列データを格納したオブジェクトを作成してみます。引数"self"は内部的に使用されるだけなので、オブジェクト生成時には残りの3つの引数を与えることになります。

In [ ]:
fasta = Fasta("gene01", "nucleotide sequence of tRNA-Ser", 
              "TGGAGTGTTGTCCGAGCGGCTGAAGGAGCATGATTGGAAATCATGTATACGGGTAAATACCTGTATCGAGGGTTCAAATCCCTCACACTCCGT")

これで先ほどの１件のFASTA形式のデータを持ったオブジェクトを生成し、fastaという変数に格納できたことになります。設計図であるクラスに対し、このように実際のデータが格納されてできたオブジェクトの実体をインスタンスと呼びます。慣例的にpythonではクラスの名称には大文字で始まる名前を用い、インスタンスに対しては小文字を使用することが推奨されています。

内部的な変数（インスタンス変数）にアクセスするには次のようにします。

In [ ]:
fasta.description

クラスの定義時には"self"という変数を使用しましたがこれは定義時に用いた仮のものなので、実際のデータにアクセスするにはインスタンスを示す変数の後ろにピリオドをつけて指定をします。また、通常の変数と同じように新たな値を代入することも可能です。

配列データ自体は通常の文字列と同じ扱いなので、次のように部分配列を取り出すことも可能です。

In [ ]:
fasta.seq[:10]

生成されたfastaがどのようなクラスに属するか確認をしてみます。

In [ ]:
type(fasta)

このように独自に定義されたクラスFastaに属するオブジェクトであることがわかります。

最後にこの配列のGC含量を求めてみます。インスタンス変数にアクセスしたときと同様にピリオドに続けてメソッド名を指定します。

In [ ]:
fasta.get_gc_content()

get_gc_contentメソッドに定義されている引数は"self"だけで、これは外部から指定しなくても自動的に自分自身を指す変数として扱われるため特に引数を与える必要はありません。したがって、この例では括弧内は空欄となっています。


## 実践的なクラスの利用

先に定義した独自クラスFastaを利用した実践的なプログラム例として、複数のFASTA形式のデータを含んだファイル（multi FASTA）からデータを１件ずつ取り出しそれぞれの配列の配列IDや長さ、GC含量等を出力するプログラムを作成してみます。

はじめにファイルを読み込んで１件ずつデータを取り出す関数を作成します。タイトル行は必ず`>`で始まるので、`>`を先頭に含んだ行が現れた時点で１件ずつデータを取り出すようにします。ただし、ファイルの先頭を読み込んだ時点では配列データが得られていませんので処理を分ける必要があります。また、ファイルを最後まで読み込んだ時点で、最後の配列データを取り出す処理を行います。


In [ ]:
def read_fasta(file_name):
    with open(file_name) as fh:
        line = next(fh)  # next関数を使い1行目（タイトル行）だけ読み込む。
        seq_id, description = line.strip(">\n").split(" ", 1)  # タイトル行を分割し始めの空白までを配列ID、空白以降をdescriptionとする。空白が含まれていない場合エラーになるので注意。
        seq = ""
        for line in fh:  # 2行目以降の読み込み
            if line.startswith(">"):  # ">"で始まる行が現れたらその時点までの配列データを返す
                yield Fasta(seq_id, description, seq)
                seq_id, description = line.strip().strip(">").split(" ", 1)  # 新たなタイトル情報を格納する
                seq = ""
            else:
                seq += line.strip("\n").upper()  # ">"で始まらない場合には、配列データを読み込んで追加していく。upperを加えたのは大文字に変換するため。
        yield Fasta(seq_id, description, seq)  # ループ終了時に最後の配列データを返す
                

ここで定義した関数read_fastaは通常の関数とは異なりジェネレーター関数と呼ばれるものです。通常の関数が呼ばれるとreturn文が現れた時点で処理を終了して値を返すのに対し、ジェネレーター関数ではyield文を用いて値を返すという違いがあります。値を返した時点でジェネレーターは一旦処理を停止してその時点での情報を内部に保ち、再度呼び出しがあると再びyield文が現れる時点まで処理を行って新たな値を返します。

ジェネレータはリストと同じようにforループと組み合わせることで１件ずつデータを取り出すことが可能です。以下が実際の使用例で、配列１件ずつを取り出して配列ID、配列の説明、長さ、GC含量をタブ区切りの文字列として出力します。fを先頭につけた文字列は"f文字列"と呼ばれるPython3.6から導入された文字列を整形するのに便利な形式で、{ }で括った中に変数名を記載することで変数の中身を文字列中に埋め込むことができます。また、変数名の後に":"で区切って各種書式を指定することも可能で、下記の例では":,d"を付けて3桁ごとにカンマで数値の桁数を区切ったり、":.1%"と付けることで小数値を百分率表記にして小数点以下の桁数を指定したりすることができます。


In [ ]:
for fasta in read_fasta("../common/s288c.fna"):
    print(f"{fasta.id}\t{fasta.description}\tLength={len(fasta.seq):,d}\tG+C%={fasta.get_gc_content():.1%}")


上記の書き方はforループを使ってリストを扱うときとほぼ同じですがリストとジェネレーターとで大きく異なることとして、リストがすべてのデータをメモリ上に保持しているのに対して、ジェネレーターでは１件ずつのデータしか保持していない点があげられます。次世代シークエンサーデータのような巨大なデータを扱う際に、１件ずつデータを取り出して処理を行うのに役立ちます。

クラスの利用例として、FASTA形式のデータを格納するためのクラス定義とmulti FASTA形式のファイルから１件ずつデータを取り出す例を紹介しました。クラスに様々なメソッドを加えることでより多様な処理を行えるように拡張可能です。たとえば、塩基配列のデータであれば相補鎖配列に変換するメソッドや、アミノ酸配列への翻訳を行うメソッドなどが考えられます。実際には、次の項で紹介するBiopythonを用いれば自分でクラス定義を行うことなく様々な機能が利用可能ですので、FASTA形式のファイルを読み込むのに自分でクラス定義を行う場面はあまり多くはないかもしれません。ただしBiopythonで読み込めないファイル形式もありますので、ここで紹介したような手法、すなわち、データを格納するための独自クラスを定義してファイルを読み込むための関数と組み合わせて処理を行うといった方法は様々な場面で応用が可能です。

__ジェネレーターについての補足__

ジェネレーターはfor文と組み合わせて用いることが多いですが、next関数を用いて１件ずつデータ取り出すことも可能です。その場合、まずはジェネレーター関数を呼び出してジェネレーターを作成します。

In [ ]:
fasta_generator = read_fasta("../common/s288c.fna")

ついでnext関数でデータを取り出します。

In [ ]:
fasta = next(fasta_generator)
print(fasta.description)

処理を繰り返すと次のデータが得られます。

In [ ]:
fasta = next(fasta_generator)
print(fasta.description)

また、list関数を組み合わせることで全データをリストに格納することができます。ただし、メモリはその分多く必要になりますので、ヒトゲノムのような巨大なサイズのデータを処理するには向きません。

In [ ]:
fasta_list = list(read_fasta("../common/s288c.fna"))
first_fasta = fasta_list[0]
last_fasta = fasta_list[-1]
print(first_fasta.description)
print(last_fasta.description)

## より高度なクラスの利用

### 特殊メソッドやstaticmethod

`__len__`を定義しておくと、len関数に引数として渡された場合の処理を定義できる。この例では配列の長さを返すようにしている。`__repr__`はオブジェクトに格納された情報の概要をprint関数やrepr関数で表示させるときの形式を定義するためのもので、おもにデバッグ用途に用いる。

In [ ]:
class Fasta:
    def __init__(self, id_, description, seq):
        self.id = id_
        self.description = description
        self.seq = seq
            
    def get_gc_content(self):
        g_count = self.seq.count("G")
        c_count = self.seq.count("C")
        gc_content = ( g_count + c_count ) / len(self.seq)
        return gc_content
    
    def __len__(self):
        return len(self.seq)
    
    def __repr__(self):
        return f"<Fasta: {self.id}, {self.description}, Length={len(self)}>"
    
    @staticmethod
    def read_fasta(file_name):
        def _parse_title(title):
            title = title.strip(">\n ")
            if " " in title:
                seq_id, description = title.split(" ", 1)
            else:
                seq_id, description = title, ""
            return seq_id, description
        
        with open(file_name) as fh:
            line = next(fh)
            seq_id, description = _parse_title(line)
            seq = ""
            for line in fh:
                if line.startswith(">"):
                    yield Fasta(seq_id, description, seq)
                    seq_id, description = _parse_title(line)
                    seq = ""
                else:
                    seq += line.strip("\n").upper()
            yield Fasta(seq_id, description, seq)

In [ ]:
fasta_list = list(Fasta.read_fasta("../common/s288c.fna"))

In [ ]:
fasta_list = sorted(fasta_list, key=len, reverse=True)

In [ ]:
for fasta in fasta_list:
    print(fasta)

### オブジェクト指向

継承、カプセル化、ポリモルフィズム等（講義では詳細省略）

# Biopythonの使い方

Biopythonは生命科学に関するデータを扱うための拡張ライブラリです。ここではBiopythonを使った配列ファイルの処理方法を扱います。


In [ ]:
# Biopythonのインポート
# SeqIOはFASTA, GenBankなどの配列データの読み書きを行うためのモジュール
from Bio import SeqIO

# FASTAファイルの読み込み

## 基本的な使い方

In [ ]:
fasta_file_name = "../common/s288c.fna"

In [ ]:
records = SeqIO.parse(fasta_file_name, "fasta")

In [ ]:
# records は ジェネレーター
type(records)

ジェネレーターはリストと同じようにfor文を使って１件ずつデータを取り出すことができますが、リストは全データをメモリに保持しているのに対し、ジェネレーターは一度に１件ずつしかデータを保持していないという違いがあります。`list(records)`とすればリストに変換することもできます。


for文を使って順次データを取り出す方法以外に、次のようにnext関数を使用する方法があります。

In [ ]:
# 一件目の配列データを取り出す
r1 = next(records)

In [ ]:
# 配列データは SeqRecord オブジェクトになっています。
# SeqRecord は配列自体の情報の他に、ID, description, アノテーションなども含んでいます。
r1

nextを繰り返し用いると以降の配列を順次取り出すことができます。前のデータに戻ることはできませんので最初に戻るには`SeqIO.parse`を繰り返し行う必要があります。

In [ ]:
print(r1)

In [ ]:
# 配列ID, descriptionなどの確認、長さは len 関数で取得できます。
print(r1.id)
print(r1.description)
print(len(r1))
print(r1.features)  # fastaファイルなのでアノテーションは含まれていない --> featuresは空

In [ ]:
# 配列を取り出します。
r1_seq = r1.seq

In [ ]:
# 配列はSeq型のオブジェクトになっています。
r1_seq

Seq型オブジェクトは部分配列の切り出し、相補鎖変換 reverse_compliment, 翻訳 translate などの機能を持ちます

In [ ]:
r1_seq[1806:2169]  # 遺伝子コード領域の切り出し
# この方法以外にも、FeatureLocationオブジェクトのextractメソッドを使用する方法もあります。(後で紹介）

In [ ]:
# 相補鎖変換
r1_seq[1806:2169].reverse_complement()

In [ ]:
# 相補鎖変換　→　翻訳
r1_seq[1806:2169].reverse_complement().translate()  

In [ ]:
# GenBank形式への変換  --> ValueError: Need a Nucleotide or Protein alphabet というエラーが出ます
print(r1.format("genbank"))

In [ ]:
# エラーを回避するには、FASTA配列の"Alphabet"を明示的に指定する必要があります。
from Bio.Alphabet import generic_dna
records = SeqIO.parse(fasta_file_name, "fasta", generic_dna)
r1 = next(records)
print(r1.format("genbank")[:1000])  # 長いので先頭の1000文字だけ表示しています

## すべてのエントリー (配列) をループで回す

In [ ]:
# recordsはジェネレータなので、リストと同じようにforループで回すことができます。
records = SeqIO.parse(fasta_file_name, "fasta")
for r in records:
    print("Seq ID=", r.id)
    print("Length=", len(r))
    print(r.seq[:50] + "...")
    print("----")

In [ ]:
# list関数を使うことでも同様のことが可能。(ファイル全体をメモリに読み込むので大きいファイルの扱いには注意)
records = list(SeqIO.parse(fasta_file_name, "fasta"))

In [ ]:
# リストにした場合には、indexを指定してデータを抽出することが可能。例えば、最後の配列を取り出すには-1を指定する。
records[-1]

__練習__  
s288c株の遺伝子タンパク質配列FASTAファイル（s288c.protein.faa）を読み込み、閾値未満の長さの配列を取り除き、降順（長いもの順）に出力するスクリプトを作成せよ。  
下記のテンプレートを使うこと。下にヒントがあります。（解答例はfasta_length_filter.pyという名称で保存しています）

In [ ]:
protein_fasta_file = "../common/s288c.protein.faa"
records = list(SeqIO.parse(protein_fasta_file, "fasta")) 
threshold = 1000

# 閾値未満のものを除く処理を追加
# recordsを降順にソートする

i = 0
for r in records:
    print(r)
    print("Length=", len(r))
    print("-----")
    i += 1
    if i == 5:
        break  # 講習用に5件出力した時点で処理を停止する。

__ヒント1__  
リスト内包表記を使ったフィルタリング

In [ ]:
# リスト内包表記を使うとリストから新しいリストを生成できる。
L = [1, 2, 3, 4, 5]
[x*x for x in L]

In [ ]:
# リストから、値が3より大きい要素を抽出
L = [1, 2, 3, 4, 5]
[x for x in L if x>3]

__ヒント２__  
keyを指定して様々な方法でソートを行う

In [ ]:
# 文字列の長さでソート
L = ["cat",  "horse", "buffalo", "dog", "ox", "hippopotamus", "tiger"]
sorted(L, key=len, reverse=True)

keyには"大小"や"順番"が判定できる値を返す任意の関数を指定することができる。自分で定義した関数を指定することもできる。  
下記はlambda式（無名関数）を使った例

In [ ]:
# lambda式を使う例。geneの後の数字部分でソート
L = ["gene_1", "gene_10", "gene_2", "gene_21", "gene_101", ]
print("普通にソート", sorted(L))
print("数字部分でソート", sorted(L, key=lambda x: int(x.split("_")[1])))

## ファイルを読み込み、辞書として格納

__本講義では詳細は割愛します__

In [ ]:
# SeqIO.to_dict関数を使うと、配列IDをkeyとした辞書にデータを格納することができます。
records = SeqIO.parse(fasta_file_name, "fasta")
dict_records = SeqIO.to_dict(records)

In [ ]:
# 全17件の辞書ができます
len(dict_records)

In [ ]:
dict_records.keys()

In [ ]:
dict_records["NC_001224.1"]

上記の方法はファイルに含まれるすべての配列情報をメモリに格納するので、大きなファイルの取り扱いには向いていません。  
別の方法としてSeqIO.index関数を使う方法もあります。

In [ ]:
idx_records = SeqIO.index(fasta_file_name, "fasta")

In [ ]:
# idx_recordsはSeq
type(idx_records)

In [ ]:
# idx_recordsは辞書と同じように使用できます。
# to_dictと異なりメモリ上にデータを保持していないため、処理速度は若干遅くなります。
idx_records.get("NC_001224.1")

In [ ]:
print(idx_records.get_raw("NC_001224.1").decode()[:300])  # get_rawで元の形式のまま取り出せます(長いので先頭300文字のみ表示しています)

## pyfaidxを使った高速な配列へのアクセス

__本講義では詳細は割愛します__

pyfaidxはFASTAファイルのインデックス（".fai"ファイル）を利用することで、FASTA形式のファイルへの高速なランダムアクセスが可能です。minicondaのデフォルトパッケージには含まれていないので、個別にインストールする必要があります。インストールはターミナルで"conda install -c bioconda pyfaidx"または"pip install faidx"を実行します。

pyfaidxの簡単な使い方は以下の通りです。詳細は公式の[ドキュメント](https://pythonhosted.org/pyfaidx/)を参考にしてください。

In [ ]:
from pyfaidx import Fasta

In [ ]:
records = Fasta(fasta_file_name)

In [ ]:
# 配列ID一覧
records.keys()

In [ ]:
# 部分配列の取り出し
record = records["NC_001133.9"]
subseq = record[100:120]
print(subseq.name)
print(subseq.seq)
print(subseq.start)
print(subseq.end)
print(subseq.fancy_name)

In [ ]:
# 相補鎖配列の取得
subseq.reverse.complement

# GenBankファイルの読み込み
FASTAファイルと同様に SeqIO.parse を利用して読み込み可能。format="genbank"を指定します。

## アノテーション情報の確認

FASTAファイルの場合と異なり、GenBank形式には配列に対する様々な注釈情報 (アノテーション) が含まれます。  
Biopythonを利用すればこれらを簡単に取得することができます。

In [ ]:
gbk_file_name = "../common/s288c.gbk"


In [ ]:
# ここではlist関数を利用して、全データをリストとして読み込んでいます。
records = list(SeqIO.parse(gbk_file_name, "genbank"))

In [ ]:
# １件目 (0番目)
r0 = records[0]

In [ ]:
print(dir(r0))

In [ ]:
# FASTAファイルを読み込んだときには空だったannotationsに配列のメタデータが辞書として格納されています。
# ここでの "annotation" は配列全体に対する生物種情報や登録者情報といったメタデータのことを示します。
# 遺伝子領域などの注釈情報は後述のfeaturesに含まれています。
r0.annotations

In [ ]:
# アノテーションされた遺伝子領域の情報はfeaturesにリストとして格納されています。(先頭10件のみ表示しています)
r0.features[:10]

In [ ]:
# 先頭(0番目)のfeatureを取り出します
f0 = r0.features[0]

In [ ]:
# 先頭のfeatureはsource featureです。
# source featureはその配列の由来について記述したもので、通常は配列1本について先頭に1件のみ記載されます。
print(f0)

In [ ]:
# 3-5番目のfeature (gene) を取り出します
f3, f4, f5 = r0.features[3:6]

In [ ]:
# featureのおもな構成要素は、type, location, qualifiers です。
# ここでは、gene, mRNA, CDS に同じ、locus_tag　が割り当てられていることから、これらが対応関係にあることがわかります。
print(f3)
print(f4)
print(f5)

下記がGenBankファイル中での５番目のfeatureについての記載
```
     CDS             complement(1807..2169)
                     /gene="PAU8"
                     /locus_tag="YAL068C"
                     /note="hypothetical protein; member of the seripauperin
                     multigene family encoded mainly in subtelomeric regions"
                     /codon_start=1
                     /product="seripauperin PAU8"
                     /protein_id="NP_009332.1"
                     /db_xref="GeneID:851229"
                     /db_xref="SGD:S000002142"
                     /translation="MVKLTSIAAGVAAIAATASATTTLAQSDERVNLVELGVYVSDIR
                     AHLAQYYMFQAAHPTETYPVEVAEAVFNYGDFTTMLTGIAPDQVTRMITGVPWYSSRL
                     KPAISSALSKDGIYTIAN"
     gene            <2480..>2707
```


以下に、上記の記載情報を取り出す方法を説明します。

In [ ]:
# qualifiersは辞書（順番情報を保持したOrderedDict）になっています。
f5.qualifiers

In [ ]:
# db_xref情報を取り出してみます。（関連するDBへの参照情報です）
# 2件の情報がリストに格納されていることがわかります。
f5.qualifiers["db_xref"]

In [ ]:
# 遺伝子産物名 (product) を確認してみます。
# 文字列として取り出すには f5.qualifiers["product"][0] などとする必要があります。
f5.qualifiers["product"]

In [ ]:
# 文字列として取り出すには f5.qualifiers["product"][0] とする必要があります。
f5.qualifiers["product"][0]

In [ ]:
# note に情報を追加してみます。
f5.qualifiers["note"].append("Test")
print(f5)

In [ ]:
# locationを確認してみます。
# 0を起点としていますので、開始位置1806は実際には1807番目の塩基を指します。
f5.location

In [ ]:
# locationオブジェクトを利用して配列の切り出しを行うことができます。
# locationのstrandが-1の場合には、自動的に相補鎖側を切り出してくれます。
# extractメソッドの引数には SeqRecord または Seq オブジェクトどちらを与えてもOKです。
f5.location.extract(r0.seq)

In [ ]:
# 配列を切り出して翻訳したものを translation qualifier の値と比較してみます。
print(str(f5.location.extract(r0.seq).translate()))
print(f5.qualifiers["translation"][0])

## ファイル全体のfeatureをループで回す

record(entry) --> feature --> qualifier の階層構造になっていることを意識してください

例としてCDSフィーチャーの中から全てのアミノ酸配列を辞書として取得する処理を実装します

In [ ]:
# すべてのアミノ酸配列を辞書として取得 (translationを含まないCDSがあったため失敗した)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            locus_tag = feature.qualifiers["locus_tag"][0]
            translation = feature.qualifiers["translation"][0]
            D[locus_tag] = translation

In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver2 (try-exceptでエラーを補足し、原因を確認)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            try:
                locus_tag = feature.qualifiers["locus_tag"][0]
                translation = feature.qualifiers["translation"][0]
                D[locus_tag] = translation
            except KeyError as e:
                # エラーが起こった場合の処理
                print(feature) # 問題のあったfeatureを表示
                raise e  # 再度、エラーを生じさせて処理を停止させる。


↑ pseudo qualifierがある場合には、translationが存在しないことがエラーの原因であった。  
そこで、pseudoの場合には処理を行わないことにする。

In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver3 (完成版)
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            if "pseudo" in feature.qualifiers:
                continue
            try:
                locus_tag = feature.qualifiers["locus_tag"][0]
                translation = feature.qualifiers["translation"][0]
                D[locus_tag] = translation
            except KeyError as e:
                print(feature.qualifiers)
                raise e


In [ ]:
# すべてのアミノ酸配列を辞書として取得 ver4 (改良版)
# get 関数を使用してkeyが存在しない場合にも対応
D = {}
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        locus_tag = feature.qualifiers.get("locus_tag", [""])[0]  # get 関数を使い locus_tag qualifierが存在しない場合には  [""] を返す
        translation = feature.qualifiers.get("translation", [""])[0]
        if locus_tag and translation: # locus_tagおよびtranslationのいずれかが空文字列のときには処理しない
            D[locus_tag] = translation

In [ ]:
# getについて
test_dict = {1:100, 2:200, 3:300}
print(test_dict.get(1))  # 100が返る
print(test_dict.get(4))  # keyが存在しない場合、None
print(test_dict.get(4, 400))  # 第２引数としてkeyが存在しない場合に返る値を指定可能。400が返る。

**[練習]**  
GenBankファイルの中身をすべてforループで辿り、アミノ酸配列(translation)・ローカスタグ(locus_tag)・遺伝子産物名(product)を取得し、  
下記のような FASTA 形式で出力するスクリプトを作成せよ。  
ただし product については空欄のものがあれば "unknown protein" とすること（getを使うと良い）。  
アミノ酸配列は `feature.qualifiers.get("translation", [""])[0]` で取得可能だが、余裕があれば`location.extract`使用すること。  
```
>YAL068C seripauperin PAU8
MVKLTSIAAGVAAIAATASATTTLAQSDERVNLVELGVYVSDIRAHLAQYYMFQAAHPTETYPVEVAEAVFNYGDFTTMLTGIAPDQVTRMITGVPWYSSRLKPAISSALSKDGIYTIAN
>YAL067W-A hypothetical protein
MPIIGVPRCLIKPFSVPVTFPFSVKKNIRILDLDPRTEAYCLSLNSVCFKRLPRRKYFHLLNSYNIKRVLGVVYC
>YAL067C putative permease SEO1
MYSIVKEIIVDPYKRLKWGFIPVKRQVEDLPDDLNSTEIVTISNSIQSHETAENFITTTSEKDQLHFETSSYSEHKDNVNVTRSYEYRDEADRPWWRFFDEQEYRINEKERSHNKWYSWFKQGTSFKEKKLLIKLDVLLAFYSCIAYWVKYLDTVNINNAYVSGMKEDLGFQGNDLVHTQVMYTVGNIIFQLPFLIYLNKLPLNYVLPSLDLCWSLLTVGAAYVNSVPHLKAIRFFIGAFEAPSYLAYQYLFGSFYKHDEMVRRSAFYYLGQYIGILSAGGIQSAVYSSLNGVNGLEGWRWNFIIDAIVSVVVGLIGFYSLPGDPYNCYSIFLTDDEIRLARKRLKENQTGKSDFETKVFDIKLWKTIFSDWKIYILTLWNIFCWNDSNVSSGAYLLWLKSLKRYSIPKLNQLSMITPGLGMVYLMLTGIIADKLHSRWFAIIFTQVFNIIGNSILAAWDVAEGAKWFAFMLQCFGWAMAPVLYSWQNDICRRDAQTRAITLVTMNIMAQSSTAWISVLVWKTEEAPRYLKGFTFTACSAFCLSIWTFVVLYFYKRDERNNAKKNGIVLYNSKHGVEKPTSKDVETLSVSDEK
...
```


解答例は"get_protein_from_gbk.py"という名称で保存してある。

In [ ]:
i = 0
for record in SeqIO.parse(gbk_file_name, "genbank"):
    for feature in record.features:
        if feature.type != "CDS":
            continue  # CDSでない場合には処理をスキップ    
        if "pseudo" in feature.qualifiers:
            continue    # pseudoである場合には処理をスキップ
        locus_tag = feature.qualifiers.get("locus_tag", [""])[0]
        translation = feature.qualifiers.get("translation", [""])[0]
        print(locus_tag)
        ### productを取得する処理を追加 ###
        i += 1
        ### print で出力する処理を追加
        if i == 5:  # 動作確認のため5件のみ出力したところで処理を停止
            break
    break # 動作確認のため最初の配列のみ処理するようにしている


# GFFファイルの読み込み

__本項は講義では扱いません__

現状ではBiopython単独ではGFFファイルの読み書きはできないため、BCBioGFFを利用する  
GFFファイルはGenBankファイルと同等の情報を保持できるが、ID-Parentを対応づけることによって明示的にgene-mRNA-CDSの階層構造を表現している。

In [ ]:
# BCBioのインポート
# GFFを扱うためのモジュール (現時点ではBiopythonには含まれていないため、別にインストールする必要がある conda install -c bioconda bcbiogff または pip install bcbio-gff）
from BCBio import GFF

In [ ]:
# ここで使うGFFファイルはRNAseq解析用に遺伝子ID(gene_id)情報を付け足したものです。ファイルの作り方については後述
gff_file_name = "../common/s288c_e.gff"

In [ ]:
records = GFF.parse(open(gff_file_name))

In [ ]:
# 最初の配列の先頭10件のfeatureを表示
# GenBankファイルの時とは異なり、mRNAやCDS featureが見えません。
# GFFファイルではmRNAはgeneの"sub_features"という扱いになっているためです。
r0 = next(records)
r0.features[:10]

In [ ]:
# 最初のgene featureを取得
gene = r0.features[4]

In [ ]:
# sub_featuresの中にmRNA featureが含まれています。
# この例ではmRNAは1件のみですが、複数のmRNAが含まれる場合もあります。
gene.sub_features

In [ ]:
# mRNA featureを取得
mrna = gene.sub_features[0]

In [ ]:
# mRNAのsub_featuresを確認
# CDS featureとexon featureが含まれています。
# sub_featureの種類はファイルによって異なります。CDSのみの場合や、非翻訳領域 (UTR) が含まれている場合もあります。
mrna.sub_features

In [ ]:
# sub_features の階層構造をたどり、どのようなfeatureが含まれているかを確認してみる。。
S=set()
for record in GFF.parse(open(gff_file_name)):
    if record.id == "NC_001224.1":
        continue  # ミトコンドリア配列は除外
    for f in record.features:
        feature_type_1 = f.type
        S.add(feature_type_1)
        for sf in f.sub_features:
            feature_type_2 = feature_type_1 + " -> " + sf.type
            S.add(feature_type_2)
            for ssf in sf.sub_features:
                feature_type_3 = feature_type_2 + " -> " + ssf.type
                S.add(feature_type_3)

    

In [ ]:
sorted(list(S), key=lambda x:(x.count("->"), x))  # 階層数でソートしたあと、アルファッベット順でソートして表示している

# 補遺

下記はこの講習で使用する各種ファイルを作成したときの手順です。

## 遺伝子IDと遺伝子産物名の対応表の作成手順

次の講義で使用予定の"gene_id_product.tsv"を作成します
各mRNAフィーチャーのproductを取得してgene_idとともに出力を行います。


In [ ]:
D = {}
for record in GFF.parse(open(gff_file_name)):
    for f in record.features:
        for sf in f.sub_features:
            if f.type == "gene" and sf.type == "mRNA":
                gene_id = sf.qualifiers["gene_id"][0]
                product = sf.qualifiers["product"][0]
                D[gene_id] = product
    

In [ ]:
# ファイルに書き出す
out_file = "../common/gene_id_product.tsv"
with open(out_file, "w") as f:
    for key, value in D.items():
        f.write(key + "\t" + value + "\n")

## GFFファイルに"gene_id"を付与するスクリプト

RNAseqのマッピング結果(BAMファイル)から、各遺伝子ごとにマップされたリード数をカウントする際にGFF形式のファイルを用いました。NCBIから取得したGFF形式のファイルに遺伝子IDとして使うのに適切な情報がなかったので、下記の方法で"gene_id"というattirbuteを加えています。

スクリプトの一部抜粋です。完成品はadd_gene_id.pyという名称でgithubの1-3ディレクトリに保存してあり、  
```python add_gene_id.py input.gff output.gff```  
とうように入力ファイル名と出力ファイル名を指定します。

```
records = list(GFF.parse(open(file_name)))

gene_cnt = 0
for r in records:
    for f in r.features:
        if f.type == "gene" or f.type == "pseudogene":
            gene_cnt += 1
            gene_id = "gene_" + str(gene_cnt).zfill(4)
            f.qualifiers["gene_id"] = [gene_id]
            for sf in f.sub_features:
                sf.qualifiers["gene_id"] = [gene_id]
                for ssf in sf.sub_features:
                    ssf.qualifiers["gene_id"] = [gene_id]

with open(out_file_name, "w") as fh:
    GFF.write(records, fh)
```